# 🔧 Preprocessing et Feature Engineering
## Santander Customer Transaction Prediction

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
import joblib
import warnings

warnings.filterwarnings('ignore')
np.random.seed(42)
print('✅ Bibliothèques importées')

✅ Bibliothèques importées


## 1️⃣ Chargement des données

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

X_train = train.drop(['ID_code', 'target'], axis=1)
y_train = train['target']
X_test = test.drop(['ID_code'], axis=1)

print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}')

X_train: (200000, 200)
y_train: (200000,)
X_test: (200000, 200)


## 2️⃣ Feature Engineering

In [3]:
def add_statistical_features(df):
    df_new = df.copy()
    df_new['mean'] = df.mean(axis=1)
    df_new['std'] = df.std(axis=1)
    df_new['min'] = df.min(axis=1)
    df_new['max'] = df.max(axis=1)
    df_new['median'] = df.median(axis=1)
    df_new['skew'] = df.skew(axis=1)
    df_new['kurt'] = df.kurtosis(axis=1)
    df_new['range'] = df_new['max'] - df_new['min']
    df_new['q1'] = df.quantile(0.25, axis=1)
    df_new['q3'] = df.quantile(0.75, axis=1)
    df_new['iqr'] = df_new['q3'] - df_new['q1']
    return df_new

X_train_eng = add_statistical_features(X_train)
X_test_eng = add_statistical_features(X_test)

print(f'Nouvelles features créées: {X_train_eng.shape[1] - X_train.shape[1]}')

Nouvelles features créées: 11


## 3️⃣ Feature Selection

In [4]:
k_best = 100
selector = SelectKBest(score_func=f_classif, k=k_best)
selector.fit(X_train_eng, y_train)

scores = pd.DataFrame({'feature': X_train_eng.columns, 'score': selector.scores_}).sort_values('score', ascending=False)
selected_features = scores.head(k_best)['feature'].tolist()

X_train_selected = X_train_eng[selected_features]
X_test_selected = X_test_eng[selected_features]

print(f'✅ {len(selected_features)} features sélectionnées')

✅ 100 features sélectionnées


## 4️⃣ Normalisation

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=selected_features, index=X_train_selected.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=selected_features, index=X_test_selected.index)

print('✅ Normalisation effectuée')

✅ Normalisation effectuée


## 5️⃣ Sauvegarde

In [6]:
joblib.dump(scaler, '../models/scaler.pkl')
print('✅ Scaler sauvegardé')

✅ Scaler sauvegardé


## ✅ Preprocessing terminé !

Prochaine étape: Modeling